## Проект "Анализ рекламных источников мобильной игры «Космические братья»".

## Описание проекта.
Вы — аналитик мобильной игры «Космические братья». Необходимо проанализировать и определить наиболее выгодные источники трафика. Составить рекомендации для отдела маркетинга.

### Ссылка на дашборд
https://public.tableau.com/app/profile/vladimir.sabirov/viz/_16822628162880/sheet2

### Ссылка на презентацию
https://drive.google.com/file/d/1gnxinYAPTnxWdJGv-vDOMlrcIRWEhCcM/view?usp=sharing

## План выполнения работы.

### Шаг 1. Ознакомление с данными и предобработка.
* Загрузка данных о событиях, источниках, стоимости привлечения.
* Изучение данных и выполнение предобработки.
* Проверка данных на наличие дубликатов и пропусков.
* Приведение типов данных во всех колонках к соответстующим сохраненным в нихзначениям.

Промежуточные выводы.

### Шаг 2. Исследовательский анализ данных.
* Определение минимальной и максимальной даты привлечения.
* Определение общего количества пользователей, пришедших из каждого источника, за весь период наблюдения.
* Определение общей стоимости кликов по каждому источнику за весь период
* Определение количества пользователей в разбикве по источникам и датам. (pivot table)
* Построение визуализации количества пользователей в разбикве по источникам и датам (heatmap)
* Определение количества активированных пользователей в разбикве по источникам и датам. (pivot table)
* Построение визуализации количества активированных пользователей в разбикве по источникам и по датам. (heatmap)
* Определение процента активированных пользователей от количества пришедших в разбивке по датам. (Конверсия)
* Определение времени активации в разбивке по датам (диаграмма boxplot).
* Определение количества построенных зданий в разбивке по их типам.
* Определение количества активированных пользователей в разбивке по методам прохождения уровня.
* Определение времени активации в разбивке по стратегиям заверешения уровня (диаграммы boxplot).
* Расчет DAU, WAU, sticky factor.
* Расчет CAC.
* (Так как у нас ограниченный по времени только первым уровнем датафрем расчет LTV, Retention Rate невозможен).
* (Для расчета ROMI в данных также недостаточно информации в связи с неизвестной суммой, которая будет приходить при каждом показе рекламы на экране постройки).

Промежуточные выводы.

### Шаг 3. Проверка статистических гипотез 
Гипотеза 1: время завершения уровня различается в зависимости от способа прохождения:
* H0: различий во времени прохождения в зависимости от стратегии прохождения нет.
* H1: различия во времени прохождения в зависимости от стратегии прохождения есть.
 
Гипотеза 2: количество построенных объектов не различается в зависимоси от канала привлечения:
* H0: количество построек не зависит от канала привлечения пользователя.
* H1: количество построек зависит от канала привлечения пользователя.

Промежуточные выводы.

### Шаг 4. Общий вывод. Рекомендации.

In [ ]:
# импортирую библиотеки
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
from scipy import stats as st 
import datetime as dt
#dateparse = lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S')
#dateparse_1 = lambda x: dt.strptime(x, '%Y-%m-%d')
import plotly.express as px
from plotly import graph_objects as go
import numpy as np
import math as mth
from scipy import stats as st

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 

## Шаг 1. Ознакомление с данными и предобработка.

In [ ]:
# загружаю данные
try:
    actions = pd.read_csv(r"D:\DOCS\datasets\Финальный проект/game_actions.csv", parse_dates=['event_datetime'])
    source = pd.read_csv(r"D:\DOCS\datasets\Финальный проект/user_source.csv")
    costs = pd.read_csv(r"D:\DOCS\datasets\Финальный проект/ad_costs.csv", parse_dates=['day'])
except FileNotFoundError:
    actions = pd.read_csv('/datasets/game_actions.csv', parse_dates=['event_datetime'])
    source = pd.read_csv('/datasets/user_source.csv')
    costs = pd.read_csv('/datasets/ad_costs.csv', parse_dates=['day'])

In [ ]:
# вывожу первые 10 строк датафрейма actions
def basic_information(x):
    display(x.head(10));
    x.info();
    print('*'*50)
    print('Дубликатов',x.duplicated().sum());
    x.drop_duplicates().reset_index(drop=True);
    print('*'*50);
    print('Количество пропусков');
    display(x.isna().sum());

Датасет *game_actions.csv*:

- `event_datetime` — время события;
- `event` — одно из трёх событий:
    1. `building` — объект построен,
    2. `finished_stage_1` — первый уровень завершён,
    3. `project` — проект завершён;
- `building_type` — один из трёх типов здания:
    1. `assembly_shop` — сборочный цех,
    2. `spaceport` — космопорт,
    3. `research_center` — исследовательский центр;
- `user_id` — идентификатор пользователя;
- `project_type` — тип реализованного проекта;

In [ ]:
basic_information(actions)

Пропуски в столбце building_type обусловлены тем, не каждый эвент это постройка, а в данном столбце отражены типы построек. В столбце project_type пропуски обусловлены тем, что игроки могли не выполнять проект во время своих игровых сессий. Данные пропуски заполнять не нужно, это может повлиять на результаты исследования.

In [ ]:
basic_information(source)

Датасет user_source.csv содержит колонки:

- `user_id` - идентификатор пользователя
- `source` - источников, с которого пришёл пользователь, установивший приложение

In [ ]:
basic_information(costs)

Датасет ad_costs.csv:

- `day` - день, в который был совершен клик по объявлению
- `source` - источник трафика
- `cost` - стоимость кликов

#### Вывод
* Получены данные из таблиц: actions (данные о событиях), source (данные об источниках трафика), costs (данные о стоимости трафика из различных источников)
* Таблица actions содержала пропуски в столбцах - building_type(тип построенного задания) - 7683 пропусков, project_type(тип выполненного проекта) - 133774 пропусков. Пропуски в столбце building_type обусловлены тем, не каждый эвент это постройка, а в данном столбце отражены типы построек. В столбце project_type пропуски обусловлены тем, что игроки могли не выполнять проект во время своих игровых сессий. Заполнение пропусков не выполнялось так как это может повлиять на результаты исследования. В таблице source  пропуски отсутствуют. В таблице costs пропуски отсутствуют.
* В таблице actions выявлен один дубликат. Так как полтеря одной строки не повлияет на результат исследвоания, его решено удалить. Таблицы sourse. costs дубликатов не содержат.
* Выполнено приведение к типу datetime столбца event_datetime таблицы actions, содержащего информацию о времени события. Также выполнено приведение к типу datetime столбца day таблицы costs, содержащего информацию о дне, в который был совершен клик по объявлению.

## Шаг 2. Исследовательский анализ данных.

#### Определение минимальной и максимальной даты привлечения.

In [ ]:
first_act_day = actions.groupby('user_id').agg({'event_datetime':'min'})
print('Минимальная дата привлечения:', first_act_day['event_datetime'].dt.date.min(), 
      'Максимальная дата привлечения:', first_act_day['event_datetime'].dt.date.max())

#### Определение общего количества пользователей, пришедших из каждого источника, за весь период наблюдения.

In [ ]:
# определил общее количество пользователей, пришедших из каждого источника, за весь период наблюдения.
source_grouped = source.groupby('source').agg({'user_id':'count'})
source_grouped = source_grouped.sort_values(by ='user_id', ascending= False)

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
source_grouped.plot(kind='bar', grid=True, figsize = (15,5), width = 0.2);
plt.xlabel('Источник привлечения', fontsize = 15)
plt.ylabel('Количество пользователей', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Количество пользователей, привлеченных в игру в разбивке по источникам', fontsize = 15)
plt.show()

По количеству привлеченных пользователей источники привлечения расположились в следующем порядке:
 * yandex direct - 4817 пользователей
 * instagram new adverts - 3347 пользователей
 * facebook_ads	- 2726 пользователей
 * youtube channel reklama - 2686 пользователей 

#### Определение общей стоимости кликов по каждому источнику за весь период

In [ ]:
# определил общую стоимость кликов по каждому источнику за весь период
costs_grouped = costs.groupby('source').agg({'cost':'sum'})
costs_grouped = costs_grouped.sort_values(by ='cost', ascending= False)

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
costs_grouped.plot(kind='bar', grid=True, figsize = (15,5), width = 0.2);
plt.xlabel('Источник привлечения', fontsize = 15)
plt.ylabel('Стоимость', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Общая стоимость кликов по каждому источнику за весь период', fontsize = 15)
plt.show()

По количеству потраченных на привлечение новых пользователей средств источники привлечения расположились в следующем порядке:
 * yandex direct - 2233.1 ₽
 * instagram new adverts - 2161.4 ₽
 * facebook_ads	- 2140.9 ₽
 * youtube channel reklama - 1068.1 ₽

#### Определение количества пользователей в разбикве по источникам и датам. (pivot table)

In [ ]:
# добавил к таблице actions источник привлечения пользователя
actions = actions.merge(source, how='left', on='user_id')

In [ ]:
# создал таблицу events в которой для каждого пользователя определена дата первой и последней сесии и источник привлечения
events_min = actions.groupby('user_id').agg({'event_datetime':'min'})
events_min.rename(columns={'event_datetime': 'first_act_day'}, inplace=True)
events_min['first_act_day'] = events_min['first_act_day'].dt.date
events_max = actions.groupby('user_id').agg({'event_datetime':'max'})
events_max.rename(columns={'event_datetime': 'last_act_day'}, inplace=True)
events_max['last_act_day'] = events_max['last_act_day'].dt.date
events = events_min.merge(events_max, how='left', on='user_id')
events.reset_index()
events = events.merge(source, how='left', on='user_id')
events

In [ ]:
# определил количества пользователей в разбикве по источникам и датам..(pivot table)
events_date_sourse_pivot = events.pivot_table(index='first_act_day', columns = 'source', values='user_id',aggfunc='count')
events_date_sourse_pivot

Наибольшее количество пользователей по всем источникам было привлечено 04.05.2020. С каждым днем количество привлеченных пользоваттелей снижалось пока не достигло самых низких значений 10.05.2020. Характер распределения по источникам сохранялся на протяжении всех дней наблюдения: yandex_direct → instagram_new_adverts → facebook_ads → youtube_channel_reklama.

#### Построение визуализации количества пользователей в разбикве по источникам и датам (heatmap)

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
events_date_sourse_pivot.plot(kind='line',grid=True,figsize=(15, 7));
plt.xlabel('Дата', fontsize = 15)
plt.ylabel('Количество пользователей', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Количество пользователей, совершивших первую активность в игре, в разбивке по дате и источнику привлечения', fontsize = 15)
plt.show()

In [ ]:
# построил хитмэп количества пользователей в разбикве по источникам и датам.
sns.set(rc = {'figure.figsize':(15,8)})
sns.heatmap(events_date_sourse_pivot, 
            annot = True, 
            fmt=".0f", 
            annot_kws={"size":13}, 
            cmap= 'coolwarm', 
            linewidths=1, 
            linecolor='black');

#### Определение количества активированных пользователей в разбикве по источникам и датам. (pivot table)

In [ ]:
# выделил из датафрейма actions пользователей, завершивших первый раунд
activ_users = actions.loc[actions['event'].values == 'finished_stage_1']
activ_users = activ_users[['user_id','event']]
# cоединил полученных пользователей, завершивших первый раунд с таблицей events по user_id
events = events.merge(activ_users, how='left', on='user_id')

In [ ]:
# заменил название колонки завершивших первый раунд на complite_round 
# со значениями 1-если пользователь завершил раунд, 0 если нет
events.rename(columns={'event': 'complite_round'}, inplace=True)
events.loc[(events.complite_round == 'finished_stage_1'), 'complite_round'] = 1
events['complite_round'] = events['complite_round'].fillna(0)
events['day_in_game'] = (events['last_act_day'] - events['first_act_day']).dt.days
events

In [ ]:
activ_users = events.loc[events['complite_round'].values == 1]
not_activ_users = events.loc[events['complite_round'].values != 1]
print('Медианное количество дней проведенных в игре у активированных пользователей равно:', activ_users['day_in_game'].median())
print('Медианное количество дней проведенных в игре у неактивированных пользователей равно:', not_activ_users['day_in_game'].median())

In [ ]:
# определил количество активированных пользователей в разбикве по источникам и датам.
activ_users_pivot = activ_users.pivot_table(index='first_act_day', columns = 'source', values='user_id',aggfunc='count')
activ_users_pivot

Распределение активированных пользователей сохранило тенденции привлеченных в игру. Максимальное количество активированных пришлось на пользователей, совершивших первое действие в игре 04.05.2020 и постепенно снижалось пропорционально количеству привлеченных пользователей, пока не достигло минимума 10.05.2020. По каналам привлечения, количество активированных пользователей распределилось в следующем порядке: yandex_direct → instagram_new_adverts → facebook_ads → youtube_channel_reklama

In [ ]:
involved_users_pivot = events.pivot_table(index='first_act_day', columns = 'source', values='user_id',aggfunc='count')
activ_users_procent_pivot = round((activ_users_pivot / involved_users_pivot *100),2)
activ_users_procent_pivot

#### Построение визуализации количества активированных пользователей в разбикве по источникам и по датам. (heatmap)

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
activ_users_pivot.plot(kind='line',grid=True,figsize=(15, 7));
plt.xlabel('Дата', fontsize = 15)
plt.ylabel('Количество пользователей', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Количество активированных пользователей в разбивке по дате первого действия и источнику привлечения', fontsize = 15)
plt.show()

In [ ]:
# построил хитмэп количества активированных пользователей в разбикве по источникам и датам.
sns.set(rc = {'figure.figsize':(15,8)})
sns.heatmap(activ_users_pivot,annot = True, 
            fmt=".0f", 
            annot_kws={"size":13}, 
            cmap= 'coolwarm', 
            linewidths=1, 
            linecolor='black');

#### Определение процента активированных пользователей от количества пришедших в разбивке по датам. (Конверсия)

In [ ]:
activ_users_group = activ_users.groupby('source').agg({'user_id':'count'})
activ_users_group = activ_users_group.merge(source_grouped, how='left', on='source')
activ_users_group.rename(columns={'user_id_y':'involved','user_id_x':'complite_stage_1'}, inplace=True)
activ_users_group

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
activ_users_group.plot(kind='bar', grid=True, figsize = (15,5), width = 0.5);
plt.xlabel('Источник привлечения', fontsize = 15)
plt.ylabel('Количество пользователей', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Количество пользователей, прошедших первый уровень, в сравнении с общим числом пользователей в разбивке по источникам', fontsize = 15)
plt.show()

In [ ]:
activ_users_group['percent'] = round((activ_users_group['complite_stage_1']/activ_users_group['involved'] *100),2)
activ_users_group

Процент активированнх пользователей от общего числа привлеченных за все время пользователей по всем каналам привлечения расположилса в диапозоне 42-44 %. По каналам процент активированных пользователей распределилсяв  следующем порядке: instagram_new_adverts → youtube_channel_reklama → yandex_direct → facebook_ads

In [ ]:
# определил процент активированных пользователей от количества пришедших в разбивке по датам и источникам. (Конверсия)
activ_percent_pivot = round((activ_users_pivot / events_date_sourse_pivot *100),1)
activ_percent_pivot 

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
activ_percent_pivot.plot(kind='line',grid=True,figsize=(15, 7));
plt.xlabel('Дата', fontsize = 15)
plt.ylabel('Процент активированнх пользователей', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Количество активированных пользователей в разбивке по дате первого действия и источнику привлечения', fontsize = 15)
plt.show()

#### Определение времени активации в разбивке по датам и источникам (диаграмма boxplot).

In [ ]:
# определил время активации в разбивке по датам (диаграмма boxplot).
# определил id пользователя и время его первого евента, считаем это за время начала активности 
first_act_time = actions.groupby('user_id').agg({'event_datetime':'min'})
first_act_time.rename(columns={'event_datetime': 'first_act'}, inplace=True)
first_act_time.reset_index()

In [ ]:
# сделал срез данных в котором хранятся id пользователей завершивших уровень,
# время завершения и источник привлечения данного пользователя
final_stage_1 = actions.loc[actions['event'].values == 'finished_stage_1']
final_stage_1 = final_stage_1[['event_datetime','user_id','source']]
final_stage_1.rename(columns={'event_datetime': 'finish_time'}, inplace=True)
final_stage_1

In [ ]:
# соединил по id таблицы с временем начала активности пользователя и временем завершения первого уровня.
# высчитал время потраченное на завершение первого уровня
activation_time = first_act_time.merge(final_stage_1, how='right', on='user_id')
activation_time['time_spent'] = activation_time['finish_time'] - activation_time['first_act']
activation_time

In [ ]:
# определил день начала активности пользователя, чтобы с дальнейшем оценить активность и время до завершения
# первого уровня в разбивке по дням привлечения
activation_time['first_act_day'] = activation_time['first_act'].dt.date
activation_time['hours_spent'] = activation_time['time_spent'].dt.total_seconds().div(3600).apply("{:g}".format)
activation_time['hours_spent'] = activation_time['hours_spent'].astype(float)
activation_time

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
sns.boxplot(data=activation_time, x='hours_spent', y='first_act_day').set(
    title='Время активации в разбивке по дате начала активности');

В среднем пользователи тратят от 250 до 285 часов на прохождение первого уровня. Важно отметить, количество сессий и их размер отследить по предоставленным данным невозможно. Реальных геймплейных часов может быть гораздо меньше. В данном случае речь идет о времени от первого действия пользователя до завершения первого уровня.

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
sns.boxplot(data=activation_time, x='hours_spent', y='source').set(title='Время активации в разбивке по источникам');

При оценке по каналам привлечения существенной разницы во времени прохождения первого уровня не выявлено. Пользователи, привлеченные со всех каналов, в среднем тратят на прохождение первого уровня около 280-285 часов.

#### Определение количества построенных зданий в разбивке по их типам.

In [ ]:
# сгруппировал количество построек для каждого пользователя
building_event = actions.loc[actions['event'].values == 'building']
building_event = building_event.groupby('user_id').agg({'event':'count'})
building_event.rename(columns={'event': 'building_count'}, inplace=True)
building_event = building_event.reset_index()
building_event = building_event.groupby('building_count').agg({'user_id':'count'})
building_event.rename(columns={'user_id': 'users_count'}, inplace=True)
#building_event = building_event.sort_values(by='users_count', ascending=False);

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
building_event.plot(kind='bar', grid=True, figsize = (15,5), width = 0.6);
plt.xlabel('Количество возведенных построек', fontsize = 15)
plt.ylabel('Количество пользователей', fontsize = 15)
plt.xticks(rotation=0)
plt.title('Количество возведенных построек в разбивке по количеству пользователей', fontsize = 15)
plt.show()

Максимальное количество пользователей за все свои игровые сессии, данные о которых содержатся в исследуемом датасете, строят зданий в порядке убывания количества пользователей 10 → 11 → 6 → 9 → 12 построек.

In [ ]:
# определил общее количество построек в разбивке по дням выполнения эвента постройка
builds = actions.loc[actions['event'].values == 'building']
pd.options.mode.chained_assignment = None
builds['event_datetime'] = builds['event_datetime'].dt.date
builds.rename(columns={'event_datetime': 'date'}, inplace=True)
builds_count = builds.groupby('date').agg({'building_type':'count'})

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
builds_count.plot(kind='bar', grid=True, width = 0.6);
plt.xlabel('Дата возведения постройки', fontsize = 15)
plt.ylabel('Количество построек', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Количество возведенных пользователями построек в разбивке по дням', fontsize = 15)
plt.show()

Общее количество построек от 04.05.2020 по 10.05.2020 увеличивается. Резко падает 11.05.2020 и постепенно снижается до конечного дня наблюдения. Данная особенность резкого снижения количества построек детально рассмотрена ниже. Спойлер - все из-за сборных цехов.

In [ ]:
events_count = actions.groupby('user_id')['event'].count()
events_count = events_count.reset_index()
events_count_1 = activation_time.merge(events_count, how='left', on='user_id')
events_count_table = events_count_1.groupby(['source','first_act_day',]).agg({'user_id':'count','event':'sum'}).reset_index()
#events_count_pivot = events_count_table.pivot_table(index='source',columns='first_act_day',values='event')

In [ ]:
events_count_pivot = events_count_table.pivot_table(index='source',columns='first_act_day',values='event')
events_count_pivot

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
events_count_pivot.plot(kind='bar',grid=True,figsize=(15, 7));
plt.xlabel('Источник', fontsize = 15)
plt.ylabel('Количество действий пользователей', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Количество действий пользователей в разбивке по дате и источнику привлечения', fontsize = 15)
plt.show()

In [ ]:
# привожу столбец day к типу datetime
costs['day'] = pd.to_datetime(costs['day'], format='%Y-%m-%d')

In [ ]:
# определил какое количество сборочных цехов пользователи строили в разбивке по датам
builds_assembly_shop = builds.loc[builds['building_type'].values == 'assembly_shop']
assembly_shop_count = builds_assembly_shop.groupby('date').agg({'building_type':'count'})
assembly_shop_count.rename(columns={'building_type': 'assembly_shop_count'}, inplace=True)
assembly_shop_count.reset_index();

Интересно, что сборочный цех, был очень популярен и пользователи стабильно его строили в начале с 04 по 10 мая, затем же пользователи полностью прекратили его строить. Возможно, мы имеем дело с каким-то техническим багом, который не дает возводить данную постройку. Так как монетизация происходит через экран показа построек баги на этм экране приведут к снижению монетизации.

In [ ]:
# определил какое количество космопотров строили пользователи в разбивке по датам
builds_spaceport = builds.loc[builds['building_type'].values == 'spaceport']
spaceport_count = builds_spaceport.groupby('date').agg({'building_type':'count'})
spaceport_count.rename(columns={'building_type': 'spaceport_count'}, inplace=True)
spaceport_count.reset_index();

Количество построенных космопортов постепенно увеличивается и становится максимальным к 12 мая. Затем постепенно снижается до даты завершения исследования.

In [ ]:
# определил какое количество исследотельских центров было построено пользователями за период наблюдения
builds_research_center = builds.loc[builds['building_type'].values == 'research_center']
research_center_count = builds_research_center.groupby('date').agg({'building_type':'count'})
research_center_count.rename(columns={'building_type': 'research_center_count'}, inplace=True)
research_center_count.reset_index();

В сравнении с космопортом исследвоательский центр - менее популярная постройка. Начинают данный тип постройки возводить только 6 мая. Максимальная активность строительства исследовательского центра, вероятно, обусловлена с другим подходом к прохождению первого уровня, а именно прохождению через выполнение исследвоания, а не через победу над врагом.

In [ ]:
# сгруппировал все типы построек по дате их возведения.
buildings_count = assembly_shop_count.merge(spaceport_count, how='right', on='date')
buildings_count = buildings_count.merge(research_center_count, how='left', on='date')
buildings_count = buildings_count.fillna(0)
buildings_count

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
buildings_count.plot(kind='bar', grid=True, width = 0.9);
plt.xlabel('Дата возведения постройки', fontsize = 15)
plt.ylabel('Количество построек', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Количество построек разных типов в разбивке по дате возведения', fontsize = 15)
plt.show()

Интересно, что сборочный цех был очень популярен и пользователи стабильно его строили в начале с 04 по 10 мая, затем же пользователи полностью прекратили его строить. Возможно, мы имеем дело с каким-то техническим багом, который не дает возводить данную постройку. Так как монетизация происходит через экран показа построек баги на этм экране приведут к снижению монетизации.

Количество построенных космопортов постепенно увеличивается и становится максимальным к 12 мая. Затем постепенно снижается до даты завершения исследования.

В сравнении с космопортом исследовательский центр - менее популярная постройка. Начинают данный тип постройки возводить только 6 мая. Максимальная активность строительства исследовательского центра, вероятно, обусловлена с другим подходом к прохождению первого уровня, а именно прохождению через выполнение исследования, а не через победу над врагом.

In [ ]:
#определил общее количество выполненных пользователями проектов в разбивке по датам
projects = actions.loc[actions['event'].values == 'project']
pd.options.mode.chained_assignment = None
projects['event_datetime'] = projects['event_datetime'].dt.date
projects.rename(columns={'event_datetime': 'date','event':'project_maker'}, inplace=True)
projects_count = projects.groupby('date').agg({'project_maker':'count'})

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
projects_count.plot(kind='bar', grid=True, width = 0.6);
plt.xlabel('Дата выполнения проекта', fontsize = 15)
plt.ylabel('Количестов выполненных проектов', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Количество выполненных пользователями проектов в разбивке по дате выполнения', fontsize = 15)
plt.show()

Количество реализованных проектов увеличивается с 06.05.2020 и достигает максимального 15.05.2020, за которым идет постепенный спад, сохраняющийся до 05.06.2020

#### Определение количества активированных пользователей в разбивке по методам прохождения уровня.

In [ ]:
finish_stage = actions[actions['event'] == 'finished_stage_1']
finish_stage

In [ ]:
# выявил количество активированных через выполнение проекта пользователей в разбивке по дате активации и каналу привлечения пользователя
finish_project = actions[actions['event'] == 'project']
finish_stage = actions[actions['event'] == 'finished_stage_1']
finish_stage_project = finish_project.query("user_id in @finish_stage.user_id")
finish_stage_project['event_datetime'] = finish_stage_project['event_datetime'].dt.date
finish_stage_project = finish_stage_project[['event_datetime','user_id','source']]
finish_stage_project_pivot = finish_stage_project.pivot_table(index='event_datetime', columns = 'source', values='user_id',aggfunc='count')

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
finish_stage_project_pivot.plot(kind='bar', grid=True,  width = 0.9);
plt.xlabel('Дата завершения проекта', fontsize = 15)
plt.ylabel('Количество пользователей', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Количество активированных через выполнение проекта пользователей в разбивке по дате активации и каналу привлечения пользователя', fontsize = 15)
plt.show()

In [ ]:
finish_stage_project_group = finish_stage_project.groupby(
    ['event_datetime',
     'source',
     ]
).agg({'user_id':'count'}).reset_index()

finish_stage_project_instagram = finish_stage_project_group[finish_stage_project_group['source'] == 'instagram_new_adverts']
finish_stage_project_facebook_ads = finish_stage_project_group[finish_stage_project_group['source'] == 'facebook_ads']
finish_stage_project_youtube = finish_stage_project_group[finish_stage_project_group['source'] == 'youtube_channel_reklama']
finish_stage_project_yandex = finish_stage_project_group[finish_stage_project_group['source'] == 'yandex_direct']

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='instagram_new_adverts', x=finish_stage_project_instagram['event_datetime'], y=finish_stage_project_instagram['user_id']),
    go.Bar(name='facebook_ads', x=finish_stage_project_facebook_ads['event_datetime'], y=finish_stage_project_facebook_ads['user_id']),
    go.Bar(name='youtube_channel_reklama', x=finish_stage_project_youtube['event_datetime'], y=finish_stage_project_youtube['user_id']),
    go.Bar(name='yandex_direct', x=finish_stage_project_yandex['event_datetime'], y=finish_stage_project_yandex['user_id']),
])

fig.update_layout(barmode='group')
fig.update_layout(title_text='Количество активированных через выполнение проекта пользователей в разбивке по дате активации и каналу привлечения пользователя', titlefont_size=12,
                  xaxis = dict(
                      title='Дата',
                      titlefont_size=12),
                  yaxis=dict(
                      title='Количество пользователей',
                      titlefont_size=12,
                            ))
fig.show()

In [ ]:
# определил количество активированных пользователей не выполнявших проект 
# в разбивке по дате активации и каналу привлечения пользователя
finish_no_project = actions[actions['event'] != 'project']
finish_stage_no_project = finish_stage.query("user_id in @finish_no_project.user_id")
finish_stage_no_project['event_datetime'] = finish_stage_no_project['event_datetime'].dt.date
finish_stage_no_project = finish_stage_no_project[['event_datetime','user_id','source']]
finish_stage_no_project_pivot = finish_stage_no_project.pivot_table(index='event_datetime', columns = 'source', values='user_id',aggfunc='count')

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
finish_stage_no_project_pivot.plot(kind='bar', grid=True,  width = 0.9);
plt.xlabel('Дата', fontsize = 15)
plt.ylabel('Количество пользователей', fontsize = 15)
plt.xticks(rotation=45)
plt.title('Количество активированных пользователей не выполнявших проект в разбивке по дате активации и каналу привлечения', fontsize = 15)
plt.show()

In [ ]:
finish_stage_no_project_group = finish_stage_no_project.groupby(
    ['event_datetime',
     'source',
     ]
).agg({'user_id':'count'}).reset_index()

finish_stage_no_project_instagram = finish_stage_no_project_group[finish_stage_no_project_group['source'] == 'instagram_new_adverts']
finish_stage_no_project_facebook_ads = finish_stage_no_project_group[finish_stage_no_project_group['source'] == 'facebook_ads']
finish_stage_no_project_youtube = finish_stage_no_project_group[finish_stage_no_project_group['source'] == 'youtube_channel_reklama']
finish_stage_no_project_yandex = finish_stage_no_project_group[finish_stage_no_project_group['source'] == 'yandex_direct']

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='instagram_new_adverts', x=finish_stage_no_project_instagram['event_datetime'], y=finish_stage_no_project_instagram['user_id']),
    go.Bar(name='facebook_ads', x=finish_stage_no_project_facebook_ads['event_datetime'], y=finish_stage_no_project_facebook_ads['user_id']),
    go.Bar(name='youtube_channel_reklama', x=finish_stage_no_project_youtube['event_datetime'], y=finish_stage_no_project_youtube['user_id']),
    go.Bar(name='yandex_direct', x=finish_stage_no_project_yandex['event_datetime'], y=finish_stage_no_project_yandex['user_id']),
])

fig.update_layout(barmode='group')
fig.update_layout(title_text='Количество активированных пользователей не выполнявших проект в разбивке по дате активации и каналу привлечения', titlefont_size=12,
                  xaxis = dict(
                      title='Дата',
                      titlefont_size=12),
                  yaxis=dict(
                      title='Количество пользователей',
                      titlefont_size=12,
                            ))
fig.show()

#### Определение времени активации в разбивке по стратегиям заверешения уровня, по источникам и дате начала активности (диаграммы boxplot).

In [ ]:
finish_stage_project_time = finish_stage_project.merge(activation_time, how='inner', on='user_id')
print('Медианное количество часов потраченных пользователями на прохождение первого уровня через реализацию провекта равно:',
      finish_stage_project_time['hours_spent'].median(),'часов, или', 
      round(((finish_stage_project_time['hours_spent'].median())/60),1),'дней')

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
sns.boxplot(data=finish_stage_project_time, x='hours_spent', y='first_act_day').set(
    title='Время активации пользователей, выполнявших проект, в разбивке по дате начала игры');

В среднем пользователи тратят на прохождение первого уровня через реализацию проекта от 310 до 370 часов.

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
sns.boxplot(data=finish_stage_project_time, x='hours_spent', y='source_x').set(
    title='Время активации пользователей, выполнявших проект, в разбивке по источнику привлечения');

В разбивке по каналам привлечения пользователи тратят на прохождение первого уровня через реализацию проекта от 310 до 320 часов

In [ ]:
finish_stage_no_project_time = finish_stage_no_project.merge(activation_time, how='inner', on='user_id')
print('Медианное количество часов потраченных пользователями на прохождение первого уровня через победу над врагом равно:',
      finish_stage_no_project_time['hours_spent'].median(),'часов, или', 
      round(((finish_stage_project_time['hours_spent'].median())/60),1),'дней')

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
sns.boxplot(data=finish_stage_no_project_time, x='hours_spent', y='first_act_day').set(
    title='Время активации пользователей, не выполнявших проект, в разбивке по дате начала игры');

В среднем пользователи тратят на прохождение первого уровня через победу над врагом от 250 до 290 часов. Что существенно меньше, чем время, потраченное на прохождение первого уровня через реализацию проекта.

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
sns.boxplot(data=finish_stage_no_project_time, x='hours_spent', y='source_x').set(
    title='Время активации пользователей, не выполнявших проект, в разбивке по источнику привлечения');

В разбивке по каналам привлечения пользователи тратят на прохождение первого уровня через победу над врагом от 260 до 270 часов. Что существенно меньше чем пользователи, которые проходят первый уровень через реализацию проекта.

#### Расчет DAU, WAU, sticky factor

In [ ]:
actions['activity_week'] = actions['event_datetime'].dt.isocalendar().week
source_wau = actions.pivot_table(index='activity_week', columns='source', values='user_id', aggfunc='nunique')

sns.set(style="whitegrid", palette="pastel", color_codes=True) 
source_wau.plot(figsize = (14,8), marker = 'o');
plt.title('Количество выполненных действий пользователей в разбивке по неделям и источникам', fontsize=17);
plt.xlabel('Неделя', fontsize = 15) 
plt.ylabel('Количество действий пользователей', fontsize = 15);

In [ ]:
actions['activity_day'] = actions['event_datetime'].dt.date
source_dau = actions.pivot_table(index='activity_day', columns='source', values='user_id', aggfunc='nunique')

sns.set(style="whitegrid", palette="pastel", color_codes=True) 
source_dau.plot(figsize = (14,8), marker = 'o');
plt.title('Количество выполненных действий пользователей в разбивке по дням и источникам', fontsize=17);
plt.xlabel('Дата', fontsize = 15) 
plt.ylabel('Количество действий пользователей', fontsize = 15);

Количество выполненных пользователями действий увеличивается с начала наблюдения 04.05.2020 по 10.05.2020. Затем происходит резкий спад активности 11.05.2020 с постепенным снижением количества действий до конца наблюдений. Вероятно, это связано с невозможностью построить очень популярный с 04.05.2020 по 10.05.2020 сборочный цех. По каналам привлечения пользователей количество выполненных действий в день снижается в следующем порядке: yandex_direct → instagram_new_adverts → facebook_ads → youtube_channel_reklama.

По каналам привлечения пользователей количество выполненных действий в неделю снижается в следующем порядке: yandex_direct → instagram_new_adverts → facebook_ads → youtube_channel_reklama.

In [ ]:
sticky_wau = actions.groupby(['activity_week','activity_day','source']).agg({'user_id':'nunique'}).reset_index()
wau_total = actions.groupby('activity_week').agg({'user_id':'nunique'}).reset_index()
sticky_wau = sticky_wau.merge(wau_total, how='inner', on='activity_week')
sticky_wau.rename(columns={'user_id_x': 'dau','user_id_y': 'wau'}, inplace=True)
sticky_wau = sticky_wau[['dau','wau','activity_week','source']]
sticky_wau['sticky_wau'] = sticky_wau['dau'] / sticky_wau['wau'] * 100
sticky_wau_pivot = sticky_wau.pivot_table(index='activity_week', columns='source', values='sticky_wau')

sns.set(style="whitegrid", palette="pastel", color_codes=True) 
sticky_wau_pivot.plot(figsize = (14,8), marker = 'o');
plt.title('STICKY WAU', fontsize=17);
plt.xlabel('Неделя', fontsize = 15) 
plt.ylabel('%', fontsize = 15);

По каналам привлечения пользователей STICKY WAU снижается в следующем порядке: yandex_direct → facebook_ads → youtube_channel_reklama → instagram_new_adverts.

#### Расчет CAC

In [ ]:
# определил общее количество пользователей, пришедших из каждого источника, за весь период наблюдения.
cac = source.groupby('source').agg({'user_id':'count'})
cac.rename(columns={'user_id': 'users_count'}, inplace=True)
cac['invest'] = costs.groupby('source').agg({'cost':'sum'})
cac['mean_cost'] =  cac['invest'] / cac['users_count']
cac = cac.sort_values(by ='mean_cost', ascending= False)
cac = cac.reset_index()

In [ ]:
sns.set(style="whitegrid", palette="pastel", color_codes=True) 
cac.plot(kind='bar', x = 'source', y = 'mean_cost', grid=True, figsize = (15,5), width = 0.2);
plt.xlabel('Источник привлечения', fontsize = 15)
plt.ylabel('Стоимость привлечения одного пользователя', fontsize = 15)
plt.xticks(rotation=45)
plt.title('CAC', fontsize = 15)
plt.show()

Стоимость привлечения пользователей с исследуемых каналов привлечения снижается в следующем порядке: facebook_ads → instagram_new_adverts → yandex_direct → youtube_channel_reklama.

В связи с хорошими показателями STICKY WAU рекомендуется увеличить привлечение пользователей с каналов yandex_direct и youtube_channel_reklama. Снизить траты на канал facebook_ads. Хуже всего показатель STICKY WAU у пользователей с канала instagram_new_adverts, а стоимость привлечения высока. Рекомендовано также снизить траты на привлечение пользователей с канала instagram_new_adverts.

#### Вывод

* Минимальная дата привлечения: 2020-05-03 Максимальная дата привлечения: 2020-05-09

* По количеству привлеченных пользователей источники привлечения расположились в следующем порядке: yandex direct - 4817 пользователей instagram new adverts - 3347 пользователей facebook_ads - 2726 пользователей youtube channel reklama - 2686 пользователей

* По количеству потраченных на привлечение новых пользователей средств источники привлечения расположились в следующем порядке: yandex direct - 2233.1 ₽ instagram new adverts - 2161.4 ₽ facebook_ads - 2140.9 ₽ youtube channel reklama - 1068.1 ₽

* Наибольшее количество пользователей по всем источникам было привлечено 04.05.2020. С каждым днем количество привлеченных пользоваттелей снижалось пока не достигло самых низких значений 10.05.2020. Характер распределения по источникам сохранялся на протяжении всех дней наблюдения: yandex_direct → instagram_new_adverts → facebook_ads → youtube_channel_reklama.

* Медианное количество дней проведенных в игре у активированных пользователей равно: 12.0. Медианное количество дней проведенных в игре у неактивированных пользователей равно: 10.0

* Распределение активированных пользователей сохранило тенденции привлеченных в игру. Максимальное количество активированных пришлось на пользователей, совершивших первое действие в игре 04.05.2020 и постепенно снижалось пропорционально количеству привлеченных пользователей, пока не достигло минимума 10.05.2020. По каналам привлечения, количество активированных пользователей распределилось в следующем порядке: yandex_direct → instagram_new_adverts → facebook_ads → youtube_channel_reklama

* Процент активированнх пользователей от общего числа привлеченных за все время пользователей по всем каналам привлечения расположилса в диапозоне 42-44 %. По каналам процент активированных пользователей распределилсяв следующем порядке: instagram_new_adverts → youtube_channel_reklama → yandex_direct → facebook_ads

* В среднем пользователи тратят от 250 до 285 часов на прохождение первого уровня. Важно отметить, количество сессий и их размер отследить по предоставленным данным невозможно ю Реальных геймплейных часов можент быть гораздо меньше. В данном случае речь идет о времени от первого действия пользователя до завершения первого уровня. При оценке по каналам привлечения существенной разницы во времени проходения первого уровня не выявлено. Пользователи, привлеченные со всех каналов, в среднем тратят на прохождение первого уровня около 280-285 часов.

* Максимальное количество пользователей за все свои игровые сессии, данные о которых содержатся в исследуемом датасете, строят зданий в порядке убывания количества пользователей 10 → 11 → 6 → 9 → 12 построек.

* Общее количество построек от 04.05.2020 по 10.05.2020 увеличивается. Резко падает 11.05.2020 и постепенно снижается до конечного дня наблюдения. Данная особенность резкого снижения количества построек детально рассмотрена ниже. Спойлер - все из- за сборных цехов.

* Интересно, что сборочный цех, был очень популярен в начале с 04 по 10 мая, затем же пользователи полность прекратили его строить. Возможно, мы имеем дело с каким-то техническим багом, который не дает возводить данную постройку. Так как монетизация происходит через экран показа построек баги на этм экране приведут к снижению монетизации. Количество построенных космопортов постепенно увеличивается и становится максимальным к 12 мая. Затем постепенно снижается до даты завершения исследования. В сравнении с космопортом исследвоательский центр - менее популярная постройка. Начинают данный тип постройки возводить только 6 мая. Максимальная активность строительства исследовательского центра, вероятно, обусловлена с другим подходом к прохождению первого уровня, а именно прохождению через выполнение исследвоания, а не через победу над врагом.

* Количество реализованных проектов увеличивается с 06.05.2020 и достигает максимального 15.05.2020 за которым идет постепенный спад сохраняющийся до 05.06.2020

* В среднем пользователи тратят на прохождение первого уровня через реализацию проекта от 310 до 370 часов. В разбивке по каналам привлечения пользователи тратят на прохождение первого уровня через реализацию проекта от 310 до 320 часов.

* В среднем пользователи тратят на прохождение первого уровня через победу над врагом от 250 до 290 часов. Что существенно меньше, чем сремя птраченное на прохождение первого уровня через реализацию проекта. В разбивке по каналам привлечения пользователи тратят на прохождение первого уровня через победу над врагом от 260 до 270 часов. Что существенно сеньше чем пользователи, которые проходят перввый уровень через реализацию проекта.

* Количество выполненных пользователями действий увеличивается с начала наблюдения 04.05.2020 по 10.05.2020. Затем происходит резкий спад активности 11.05.2020 с постепенным снижением количества действий до конца наблюдений. Вероятно, это связано с невозможностью построить очень популярный с 04.05.2020 по 10.05.2020 сборочный цех. По каналам привлечения пользователей количество выполненных действий в день снижается в следующем порядке: yandex_direct → instagram_new_adverts → facebook_ads → youtube_channel_reklama. По каналам привлечения пользователей количество выполненных действий в неделю снижается в следующем порядке: yandex_direct → instagram_new_adverts → facebook_ads → youtube_channel_reklama.

* По каналам привлечения пользователей STICKY WAU снижается в следующем порядке: yandex_direct → facebook_ads → youtube_channel_reklama → instagram_new_adverts.

* Стоимость привлечения пользователей с исследуемых каналов привлечения снижается в следующем порядке: facebook_ads → instagram_new_adverts → yandex_direct → youtube_channel_reklama.

* В связи с хорошими показателями STICKY WAU рекомендуется увеичить привлечение пользователей с каналов yandex_direct и youtube_channel_reklama. Снизить траты на канал facebook_ads. Хуже всего показатель STICKY WAU у пользователей с канала instagram_new_adverts, а стоимость привлечения высока. Рекомендовано также снизить траты на првлечения пользователей с канала instagram_new_adverts.

## Шаг 3. Проверка статистических гипотез

In [ ]:
# сделано два среза данных пользователей прошедших первый уровень. первый срез id пользователей и потраченное количество
# часов на прохождение уровня через реализацию проекта, второй срез - id пользователей и потраченное количество
# часов на прохождение уровня через победу над врагом
progect = finish_stage_project_time.groupby('hours_spent').agg({'user_id':'unique'})
progect = progect.reset_index()
progect = progect['hours_spent'].astype(int)
no_progect = finish_stage_no_project_time.groupby('hours_spent').agg({'user_id':'unique'})
no_progect = no_progect.reset_index()
no_progect = no_progect['hours_spent'].astype(int)

#### Считаю статистическую значимость различий  во времени прохождения уровня с зависимотси от стратегии завершения уровня(победа над врагом или выполнение проекта):

  Сформулируем гипотезы:
  
* H0: различий во времени прохождения в зависимости от стратегии прохождения нет.
* H1: различия во времени прохождения в зависимости от стратегии прохождения есть.

In [ ]:
alpha = .05 # критический уровень статистической значимости

results = st.ttest_ind(progect, no_progect)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Ранее было установлено, что пользователи проходящие первый уровень через победу над врагом тратят в среднем меньше времени на прохождение в сравнении с пользователями, выполняющими проект. Данный тезис подтвержден статистическим тестом.

In [ ]:
# сделал срез данных всех эвентов постройка. Сгруппировал по источнику и id пользователя
sourse_user_build = actions.loc[actions['event'].values == 'building']
sourse_user_build = sourse_user_build.groupby(['source','user_id']).agg({'event':'count'})
sourse_user_build = sourse_user_build.reset_index()

In [ ]:
# выделил в отдельные таблицы количество построек выполненных пользователями в зависимости от канала привлечения
facebook_ads_builds = sourse_user_build[sourse_user_build['source'] == 'facebook_ads']
facebook_ads_builds = facebook_ads_builds['event']
facebook_ads_builds = facebook_ads_builds.reset_index(drop=True)

instagram_new_adverts_builds = sourse_user_build[sourse_user_build['source'] == 'instagram_new_adverts']
instagram_new_adverts_builds = instagram_new_adverts_builds['event']
instagram_new_adverts_builds = instagram_new_adverts_builds.reset_index(drop=True)

yandex_direct_builds = sourse_user_build[sourse_user_build['source'] == 'yandex_direct']
yandex_direct_builds = yandex_direct_builds['event']
yandex_direct_builds = yandex_direct_builds.reset_index(drop=True)

youtube_channel_reklama_builds = sourse_user_build[sourse_user_build['source'] == 'youtube_channel_reklama']
youtube_channel_reklama_builds = youtube_channel_reklama_builds['event']
youtube_channel_reklama_builds = youtube_channel_reklama_builds.reset_index(drop=True)

#### Считаю статистическую значимость различий  в количестве построенных объектов в зависимости от канала привлечения:

  Сформулируем гипотезы:
  
* H0: количество построек не зависит от канала привлечения пользователя.
* H1: количество построек зависит от канала привлечения пользователя.

In [ ]:
def z_test(H0, H1, title):
    print(title)

    results = st.mannwhitneyu(H0, H1)

    print('p-значение: ', results.pvalue)
    
    bonferroni_alpha =   alpha / 6

    if results.pvalue < bonferroni_alpha:
        print('Отвергаем нулевую гипотезу: разница статистически значима')
    else:
        print(
            'Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя'
        )

In [ ]:
z_test(facebook_ads_builds, instagram_new_adverts_builds, 'facebook/instagram')
print('************************')
z_test(facebook_ads_builds, yandex_direct_builds, 'facebook/yandex')
print('************************')
z_test(facebook_ads_builds, youtube_channel_reklama_builds, 'facebook/youtube')
print('************************')
z_test(instagram_new_adverts_builds, yandex_direct_builds, 'instagram/yandex')
print('************************')
z_test(instagram_new_adverts_builds, youtube_channel_reklama_builds, 'instagram/youtube')
print('************************')
z_test(yandex_direct_builds, youtube_channel_reklama_builds, 'yandex/youtube')

Ни в одной из пар сравнений каналов привлечения не удалось получить статистически достоверных различий в количестве выполненных построек. Можно сделать вывод о том, что количество построек не зависит от канала привлечения.

#### Вывод:
* При проверке статистической значимости гипотезы о различии времени прохождения уровня в зависимотси от стратегии завершения уровня(победа над врагом или выполнение проекта). Выявлены различия во времени прохождения в зависимости от стратегии прохождения. Ранее было установлено, что пользователи проходящие первый уровень через победу над врагом тратят в среднем меньше времени на прохождение в сравнении с пользователями, выполняющими проект. Данный тезис подтвержден статистическим тестом.

* При проверке статистической значимости гипотезы о различиях в количестве построенных объектов в зависимости от канала привлечения ни в одной из пар сравнений каналов привлечения не удалось получить статистически достоверных различий в количестве выполненных построек. Можно сделать вывод о том, что количество построек не зависит от канала привлечения.

### Шаг 4. Общий вывод. Рекомендации.¶

#### Вывод по шагу 1 "Ознакомление с данными и предобработка":

* Получены данные из таблиц: actions (данные о событиях), source (данные об источниках трафика), costs (данные о стоимости трафика из различных источников)

* Таблица actions содержала пропуски в столбцах - building_type(тип построенного задания) - 7683 пропусков, project_type(тип выполненного проекта) - 133774 пропусков. Пропуски в столбце building_type обусловлены тем, не каждый эвент это постройка, а в данном столбце отражены типы построек. В столбце project_type пропуски обусловлены тем, что игроки могли не выполнять проект во время своих игровых сессий. Заполнение пропусков не выполнялось так как это может повлиять на результаты исследования. В таблице source  пропуски отсутствуют. В таблице costs пропуски отсутствуют.

* В таблице actions выявлен один дубликат. Так как полтеря одной строки не повлияет на результат исследвоания, его решено удалить. Таблицы sourse. costs дубликатов не содержат.

* Выполнено приведение к типу datetime столбца event_datetime таблицы actions, содержащего информацию о времени события. Также выполнено приведение к типу datetime столбца day таблицы costs, содержащего информацию о дне, в который был совершен клик по объявлению.

#### Вывод по шагу 2 "Исследовательский анализ данных":

* Минимальная дата привлечения: 2020-05-03 Максимальная дата привлечения: 2020-05-09

* По количеству привлеченных пользователей источники привлечения расположились в следующем порядке: yandex direct - 4817 пользователей instagram new adverts - 3347 пользователей facebook_ads - 2726 пользователей youtube channel reklama - 2686 пользователей

* По количеству потраченных на привлечение новых пользователей средств источники привлечения расположились в следующем порядке: yandex direct - 2233.1 ₽ instagram new adverts - 2161.4 ₽ facebook_ads - 2140.9 ₽ youtube channel reklama - 1068.1 ₽

* Наибольшее количество пользователей по всем источникам было привлечено 04.05.2020. С каждым днем количество привлеченных пользоваттелей снижалось пока не достигло самых низких значений 10.05.2020. Характер распределения по источникам сохранялся на протяжении всех дней наблюдения: yandex_direct → instagram_new_adverts → facebook_ads → youtube_channel_reklama.

* Медианное количество дней проведенных в игре у активированных пользователей равно: 12.0. Медианное количество дней проведенных в игре у неактивированных пользователей равно: 10.0

* Распределение активированных пользователей сохранило тенденции привлеченных в игру. Максимальное количество активированных пришлось на пользователей, совершивших первое действие в игре 04.05.2020 и постепенно снижалось пропорционально количеству привлеченных пользователей, пока не достигло минимума 10.05.2020. По каналам привлечения, количество активированных пользователей распределилось в следующем порядке: yandex_direct → instagram_new_adverts → facebook_ads → youtube_channel_reklama

* Процент активированнх пользователей от общего числа привлеченных за все время пользователей по всем каналам привлечения расположилса в диапозоне 42-44 %. По каналам процент активированных пользователей распределилсяв следующем порядке: instagram_new_adverts → youtube_channel_reklama → yandex_direct → facebook_ads

* В среднем пользователи тратят от 250 до 285 часов на прохождение первого уровня. Важно отметить, количество сессий и их размер отследить по предоставленным данным невозможно ю Реальных геймплейных часов можент быть гораздо меньше. В данном случае речь идет о времени от первого действия пользователя до завершения первого уровня. При оценке по каналам привлечения существенной разницы во времени проходения первого уровня не выявлено. Пользователи, привлеченные со всех каналов, в среднем тратят на прохождение первого уровня около 280-285 часов.

* Максимальное количество пользователей за все свои игровые сессии, данные о которых содержатся в исследуемом датасете, строят зданий в порядке убывания количества пользователей 10 → 11 → 6 → 9 → 12 построек.

* Общее количество построек от 04.05.2020 по 10.05.2020 увеличивается. Резко падает 11.05.2020 и постепенно снижается до конечного дня наблюдения. Данная особенность резкого снижения количества построек детально рассмотрена ниже. Спойлер - все из- за сборных цехов.

* Интересно, что сборочный цех, был очень популярен в начале с 04 по 10 мая, затем же пользователи полность прекратили его строить. Возможно, мы имеем дело с каким-то техническим багом, который не дает возводить данную постройку. Так как монетизация происходит через экран показа построек баги на этм экране приведут к снижению монетизации. Количество построенных космопортов постепенно увеличивается и становится максимальным к 12 мая. Затем постепенно снижается до даты завершения исследования. В сравнении с космопортом исследвоательский центр - менее популярная постройка. Начинают данный тип постройки возводить только 6 мая. Максимальная активность строительства исследовательского центра, вероятно, обусловлена с другим подходом к прохождению первого уровня, а именно прохождению через выполнение исследвоания, а не через победу над врагом.

* Количество реализованных проектов увеличивается с 06.05.2020 и достигает максимального 15.05.2020 за которым идет постепенный спад сохраняющийся до 05.06.2020

* В среднем пользователи тратят на прохождение первого уровня через реализацию проекта от 310 до 370 часов. В разбивке по каналам привлечения пользователи тратят на прохождение первого уровня через реализацию проекта от 310 до 320 часов.

* В среднем пользователи тратят на прохождение первого уровня через победу над врагом от 250 до 290 часов. Что существенно меньше, чем сремя птраченное на прохождение первого уровня через реализацию проекта. В разбивке по каналам привлечения пользователи тратят на прохождение первого уровня через победу над врагом от 260 до 270 часов. Что существенно сеньше чем пользователи, которые проходят перввый уровень через реализацию проекта.

* Количество выполненных пользователями действий увеличивается с начала наблюдения 04.05.2020 по 10.05.2020. Затем происходит резкий спад активности 11.05.2020 с постепенным снижением количества действий до конца наблюдений. Вероятно, это связано с невозможностью построить очень популярный с 04.05.2020 по 10.05.2020 сборочный цех. По каналам привлечения пользователей количество выполненных действий в день снижается в следующем порядке: yandex_direct → instagram_new_adverts → facebook_ads → youtube_channel_reklama. По каналам привлечения пользователей количество выполненных действий в неделю снижается в следующем порядке: yandex_direct → instagram_new_adverts → facebook_ads → youtube_channel_reklama.

* По каналам привлечения пользователей STICKY WAU снижается в следующем порядке: yandex_direct → facebook_ads → youtube_channel_reklama → instagram_new_adverts.

* Стоимость привлечения пользователей с исследуемых каналов привлечения снижается в следующем порядке: facebook_ads → instagram_new_adverts → yandex_direct → youtube_channel_reklama.

* В связи с хорошими показателями STICKY WAU рекомендуется увеичить привлечение пользователей с каналов yandex_direct и youtube_channel_reklama. Снизить траты на канал facebook_ads. Хуже всего показатель STICKY WAU у пользователей с канала instagram_new_adverts, а стоимость привлечения высока. Рекомендовано также снизить траты на првлечения пользователей с канала instagram_new_adverts.

#### Вывод по шагу 3 "Проверка статистических гипотез":

* При проверке статистической значимости гипотезы о различии времени прохождения уровня в зависимотси от стратегии завершения уровня(победа над врагом или выполнение проекта). Выявлены различия во времени прохождения в зависимости от стратегии прохождения. Ранее было установлено, что пользователи проходящие первый уровень через победу над врагом тратят в среднем меньше времени на прохождение в сравнении с пользователями, выполняющими проект. Данный тезис подтвержден статистическим тестом.
* При проверке статистической значимости гипотезы о различиях в количестве построенных объектов в зависимости от канала привлечения ни в одной из пар сравнений каналов привлечения не удалось получить статистически достоверных различий в количестве выполненных построек. Можно сделать вывод о том, что количество построек не зависит от канала привлечения.